In [5]:
import numpy as np

def monthday_to_days(month, day):
    month = np.asarray(month, dtype=int)
    day = np.asarray(day, dtype=int)
    month_days = np.array([31,28,31,30,31,30,31,31,30,31,30,31], dtype=int)  # 2017
    month_starts = np.concatenate(([0], np.cumsum(month_days)[:-1]))
    return month_starts[month - 1] + (day - 1)   # enero 1 -> 0

def moving_average(x, window=30):
    x = np.asarray(x, dtype=float)
    w = np.ones(window, dtype=float) / window
    return np.convolve(x, w, mode="same")

def iqr_outlier_mask(x, k=1.5):
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    low = q1 - k * iqr
    high = q3 + k * iqr
    return (x < low) | (x > high)

def load_and_preprocess(path="kumpula-weather-2017.csv"):
    data = np.genfromtxt(path, delimiter=",", names=True, dtype=None, encoding="utf-8")

    month = data["m"].astype(int)
    day = data["d"].astype(int)
    temp = data["Air_temperature_degC"].astype(float)

    dias = monthday_to_days(month, day).astype(float)

    # limpiar NaN/inf
    valid = np.isfinite(dias) & np.isfinite(temp)
    dias = dias[valid]
    temp = temp[valid]

    # ordenar por día
    idx = np.argsort(dias)
    dias = dias[idx]
    temp = temp[idx]

    # anomalías
    ma30 = moving_average(temp, window=30)
    anomalies = temp - ma30

    # outliers 
    outliers = iqr_outlier_mask(anomalies, k=1.5)

    temps_clean = np.column_stack([temp, anomalies])  # (N,2)
    return temps_clean, dias, outliers

temps_clean, dias, outlier_mask = load_and_preprocess("kumpula-weather-2017.csv")

print("temps_clean shape:", temps_clean.shape)  # (N,2)
print("dias shape:", dias.shape)                # (N,)
print("outliers detectados:", int(outlier_mask.sum()))

temps_clean shape: (365, 2)
dias shape: (365,)
outliers detectados: 8


<div style="background-color:#ffe6f0; padding:15px; border-radius:10px; border:1px solid #ffb3d9">

🌸 **Ejercicio 1 - Procesamiento de datos**

Se cargaron los datos de temperatura y se transformaron las fechas (mes y día) a días del año para facilitar el análisis temporal.

Luego, se limpiaron los datos eliminando valores inválidos y se ordenaron cronológicamente. Se calculó un promedio móvil de 30 días para obtener la tendencia de la temperatura y, a partir de este, se determinaron las anomalías.

Finalmente, se identificaron outliers mediante el método del rango intercuartílico (IQR), permitiendo detectar variaciones anómalas en la serie.

</div>

In [6]:
import numpy as np
import pandas as pd
from functools import wraps


df = pd.read_csv("kumpula-weather-2017.csv")

month = pd.to_numeric(df["m"], errors="coerce").to_numpy()
day   = pd.to_numeric(df["d"], errors="coerce").to_numpy()
temp  = pd.to_numeric(df["Air temperature (degC)"], errors="coerce").to_numpy()

# month-day 
month_days = np.array([31,28,31,30,31,30,31,31,30,31,30,31], dtype=int)
month_starts = np.concatenate(([0], np.cumsum(month_days)[:-1]))
dias = month_starts[month.astype(int) - 1] + (day.astype(int) - 1)

# limpiar NaN/inf
valid = np.isfinite(dias) & np.isfinite(temp)
dias = dias[valid].astype(float)
temp = temp[valid].astype(float)

# anomalía = temp - media móvil 30 días
mov30 = np.convolve(temp, np.ones(30)/30, mode="same")
anom = temp - mov30

temps_clean = np.column_stack([temp, anom])

print("dias shape:", dias.shape)
print("temps_clean shape:", temps_clean.shape)


# clases + herencia + decorator


def vectorize(func):
    @wraps(func)
    def wrapper(self, x, *args, **kwargs):
        x = np.asarray(x)
        vf = np.vectorize(lambda z: func(self, z, *args, **kwargs))
        return vf(x)
    return wrapper

class TimeSeriesAnalyzer:
    def __init__(self, t, y):
        self.t = np.asarray(t, dtype=float)
        self.y = np.asarray(y, dtype=float)

    def smooth(self, window=7):
        w = np.ones(window, dtype=float) / window
        return np.convolve(self.y, w, mode="same")

    @vectorize
    def identity(self, x):
        return x

class WeatherAnalyzer(TimeSeriesAnalyzer):
    def seasonal_decompose(self):
        # trend: polyfit deg=2
        c2 = np.polyfit(self.t, self.y, deg=2)
        trend = np.polyval(c2, self.t)

    
        y_detr = self.y - trend
        n = len(y_detr)
        fhat = np.fft.rfft(y_detr)
        amp = np.abs(fhat)
        if len(amp) > 0:
            amp[0] = 0.0

        k = min(4, len(fhat))
        idx = np.argsort(amp)[-k:]

        keep = np.zeros_like(fhat, dtype=complex)
        keep[idx] = fhat[idx]
        seasonal = np.fft.irfft(keep, n=n)

        residual = self.y - trend - seasonal
        return trend, seasonal, residual

    def forecast(self, days_ahead=30):
        
        m = min(30, len(self.t))
        t_last = self.t[-m:]
        y_last = self.y[-m:]

        c3 = np.polyfit(t_last, y_last, deg=3)
        t_future = np.arange(self.t[-1] + 1, self.t[-1] + days_ahead + 1)

        y_poly = np.polyval(c3, t_future)

        # ruido según residuos del ajuste local
        resid = y_last - np.polyval(c3, t_last)
        sigma = np.std(resid)
        np.random.seed(1997)
        noise = np.random.normal(0.0, sigma, size=days_ahead)

        y_future = y_poly + noise
        return t_future, y_future


analyzer = WeatherAnalyzer(dias, temps_clean[:, 1])

smooth7 = analyzer.smooth(window=7)
trend, seasonal, residual = analyzer.seasonal_decompose()
future_days, future_vals = analyzer.forecast(days_ahead=30)

print("smooth7 shape:", smooth7.shape)
print("trend shape:", trend.shape)
print("seasonal shape:", seasonal.shape)
print("residual shape:", residual.shape)
print("forecast shapes:", future_days.shape, future_vals.shape)

assert temps_clean.shape[1] == 2
assert len(dias) == len(temps_clean)
assert trend.shape == seasonal.shape == residual.shape == dias.shape
assert future_days.shape == future_vals.shape == (30,)
print("Todo OK ✅")

dias shape: (365,)
temps_clean shape: (365, 2)
smooth7 shape: (365,)
trend shape: (365,)
seasonal shape: (365,)
residual shape: (365,)
forecast shapes: (30,) (30,)
Todo OK ✅


<div style="background-color:#ffe6f0; padding:18px; border-radius:12px; border:1px solid #f1aeb5">

### 🌸 Ejercicio2 — Advanced Class with Inheritance/Decorators

Cargue la serie de temperatura e implemente una estructura basada en clases:

- `TimeSeriesAnalyzer` con suavizado (*moving average*).
- Decorador `@vectorize` para operar por columnas.
- `WeatherAnalyzer` con:
  - ✨ `seasonal_decompose()` (trend, seasonal y residual).
  - 🔮 `forecast()` usando ajuste polinomial y ruido.

Se verificó que todas las salidas tuvieran dimensiones correctas (`dias`, `temps_clean`, `trend`, `seasonal`, `residual`, `forecast`).

**Conclusión**

El modelo funciona correctamente y cumple con los objetivos del ejercicio.

</div>

In [7]:
import numpy as np
import pandas as pd

# ejercicio 3: Robust I/O + Processed

if "dias" not in globals() or "temps_clean" not in globals():
    # fallback mínimo: cargar desde archivo local
    df = pd.read_csv("kumpula-weather-2017.csv")
    month = pd.to_numeric(df["m"], errors="coerce").to_numpy()
    day   = pd.to_numeric(df["d"], errors="coerce").to_numpy()
    temp  = pd.to_numeric(df["Air temperature (degC)"], errors="coerce").to_numpy()

    month_days = np.array([31,28,31,30,31,30,31,31,30,31,30,31])
    month_starts = np.concatenate(([0], np.cumsum(month_days)[:-1]))
    dias = month_starts[month.astype(int)-1] + (day.astype(int)-1)

    valid = np.isfinite(dias) & np.isfinite(temp)
    dias = dias[valid].astype(float)
    temp = temp[valid].astype(float)

    ma30 = np.convolve(temp, np.ones(30)/30, mode="same")
    anomalies = temp - ma30
    temps_clean = np.column_stack([temp, anomalies])

# Variables principales
days = np.asarray(dias, dtype=float)
temp = np.asarray(temps_clean[:, 0], dtype=float)
anomaly = np.asarray(temps_clean[:, 1], dtype=float)

# smooth para subset
temp_smooth = np.convolve(temp, np.ones(7)/7, mode="same")


np.savez("processed_weather.npz", temps_clean=temps_clean, anomalies=anomaly, days=days)
print("Guardado: processed_weather.npz")


n = min(100, len(days))
subset = np.column_stack([days[:n], temp_smooth[:n], anomaly[:n]])

np.savetxt(
    "subset.csv",
    subset,
    delimiter=",",
    fmt="%.2f",
    header="days,temp_smooth,anomaly",
    comments=""
)
print("Guardado: subset.csv")

def load_and_validate(filename):
    out = {"filename": filename, "ok": False, "format": None, "arrays": {}}

    if filename.endswith(".npz"):
        out["format"] = "npz"
        data = np.load(filename)

        for k in data.files:
            arr = np.asarray(data[k])
            out["arrays"][k] = arr

        # Validaciones mínimas
        if "temps_clean" not in out["arrays"]:
            raise ValueError("NPZ inválido: falta 'temps_clean'.")

        tc = out["arrays"]["temps_clean"]
        if tc.ndim != 2 or tc.shape[1] != 2:
            raise ValueError(f"'temps_clean' inválido: shape esperado (N,2), recibido {tc.shape}.")

        for k, arr in out["arrays"].items():
            if np.issubdtype(arr.dtype, np.number):
                if np.isnan(arr).any() or np.isinf(arr).any():
                    raise ValueError(f"'{k}' contiene NaN o Inf.")

        out["ok"] = True
        return out

    elif filename.endswith(".csv"):
        out["format"] = "csv"
        arr = np.loadtxt(filename, delimiter=",", skiprows=1)
        if arr.ndim == 1:
            arr = arr.reshape(1, -1)

        # subset.csv debe ser (N,3)
        if arr.shape[1] != 3:
            raise ValueError(f"CSV inválido: se esperaban 3 columnas, recibidas {arr.shape[1]}.")

        if np.isnan(arr).any() or np.isinf(arr).any():
            raise ValueError("CSV contiene NaN o Inf.")

        out["arrays"]["table"] = arr
        out["ok"] = True
        return out

    else:
        raise ValueError("Formato no soportado. Usa .npz o .csv")

info_npz = load_and_validate("processed_weather.npz")
info_csv = load_and_validate("subset.csv")

print("NPZ válido:", info_npz["ok"], "| keys:", list(info_npz["arrays"].keys()))
print("CSV válido:", info_csv["ok"], "| shape:", info_csv["arrays"]["table"].shape)

Guardado: processed_weather.npz
Guardado: subset.csv
NPZ válido: True | keys: ['temps_clean', 'anomalies', 'days']
CSV válido: True | shape: (100, 3)


<div style="background-color:#ffe6f0; padding:18px; border-radius:12px; border:1px solid #f1aeb5">

### 🌸 Ejercicio 3 — Robust I/O + Processed

Se guardaron los datos procesados en:

- `processed_weather.npz`, incluyendo las variables `temps_clean`, `anomalies` y `days`.
- `subset.csv`, con los primeros 100 días y las columnas `days`, `temp_smooth` y `anomaly`, utilizando formato `%.2f`.

Implementamos la función `load_and_validate(filename)` para:

- Cargar archivos en formato `.npz` o `.csv`.
- Verificar la forma de los datos.
- Comprobar la ausencia de valores `NaN` e `Inf`.

La validación fue correcta en ambos archivos (`True`), y el archivo `subset.csv` presentó dimensiones consistentes de `(100, 3)`.

</div>

<div style="background-color:#ffe6f0; padding:18px; border-radius:12px; border:1px solid #ffb3d1; font-family: 'Segoe UI', sans-serif;">

<h3 style="color:black;">💗 Preguntas 1. Fracción de outliers (IQR) y tipo de método:</h3>
<p style="color:#444;">
La fracción removida es  
\( 1 - \frac{N_{\text{filtrados}}}{N_{\text{total}}} \).  
El método IQR es robusto (no asume normalidad), generalmente conservador.  
Alternativa: usar \( \mu \pm 3\sigma \) (método 3-sigma).
</p>

<hr>

<h3 style="color:black;">💗 2. Ejecución de @vectorize (llamadas y forma):</h3>
<p style="color:#444;">
Para <code>data.shape = (365,2)</code>, <code>smooth</code> se llama 2 veces (una por columna).  
<code>results.T</code> se usa para recuperar la forma original (365,2).      
Alternativa: <code>np.apply_along_axis</code>.
</p>

<hr>

<h3 style="color:black;">💗 3. Uso de FFT en seasonal_decompose:</h3>
<p style="color:#444;">
Se resta la tendencia para aislar componentes periódicas.  
Las top-4 frecuencias representan los ciclos dominantes (ej. semanal, mensual, anual).
</p>

<hr>

<h3 style="color:black;">💗 4. Comparación stats (raw vs smooth):</h3>
<p style="color:#444;">
La desviación estándar disminuye al suavizar porque se reduce el ruido.  
<code>polyfit</code> grado 2 captura bien la tendencia por su curvatura suave sin sobreajuste.
</p>

</div>